# Model training

In [9]:
## import 
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.metrics import confusion_matrix, classification_report, roc_auc_score
from sklearn.metrics import accuracy_score

loading dataframe

In [7]:
df = pd.read_csv('../data/processed/cleaned_cybersecurity_attacks_encoded.csv', index_col= 'Attack_Index')
df.head()


,Packet Length,Anomaly Scores,Attack Type,Severity Level,Protocol_ICMP,Protocol_TCP,Protocol_UDP,Packet Type_Control,Packet Type_Data,Traffic Type_DNS,...,Action Taken_Logged.1,Network Segment_Segment A.1,Network Segment_Segment B.1,Network Segment_Segment C.1,Firewall Logs_Log Data.1,Firewall Logs_No Log Data.1,IDS/IPS Alerts_Alert Data.1,IDS/IPS Alerts_No Alert Data.1,Log Source_Firewall.1,Log Source_Server.1
Attack_Index,,,,,,,,,,,,,,,,,,,,,
0,503,28.67,Malware,1,1,0,0,0,1,0,...,1,1,0,0,1,0,0,1,0,1
1,1174,51.50,Malware,1,1,0,0,0,1,0,...,0,0,1,0,1,0,0,1,1,0
2,306,87.42,DDoS,1,0,0,1,1,0,0,...,0,0,0,1,1,0,1,0,1,0
3,385,15.79,Malware,2,0,0,1,0,1,0,...,0,0,1,0,0,1,1,0,1,0
4,1462,0.52,DDoS,1,0,1,0,0,1,1,...,0,0,0,1,0,1,1,0,1,0


## Dataset Splitting

In [12]:
df_train, df_test = train_test_split(df,test_size = 0.2, random_state=42)
df_train.shape, df_test.shape

((32000, 56), (8000, 56))

let see the distribution 

In [13]:
print(df['Severity Level'].mean())
print(df_train['Severity Level'].mean())
print(df_test['Severity Level'].mean())

1.001325
0.9975625
1.016375


### Dividing the target column and feature columns

In [14]:
# list of columns features to be used for training
features = [
    'Packet Length', 
    'Anomaly Scores', 
    'Severity Level', 
    'Protocol_ICMP',
    'Protocol_TCP', 
    'Protocol_UDP', 
    'Packet Type_Control',
    'Packet Type_Data', 
    'Traffic Type_DNS', 
    'Traffic Type_FTP',
    'Traffic Type_HTTP', 
    'Malware Indicators_IoC Detected',
    'Malware Indicators_No IoC Detected', 
    'Alerts/Warnings_Alert Triggered',
    'Alerts/Warnings_No Alert Triggered', 
    'Attack Signature_Known Pattern A', 
    'Attack Signature_Known Pattern B',
    'Action Taken_Blocked', 
    'Action Taken_Ignored',
    'Action Taken_Logged',
    'Network Segment_Segment A', 
    'Network Segment_Segment B',
    'Network Segment_Segment C',
    'Firewall Logs_Log Data',
    'Firewall Logs_No Log Data',
    'IDS/IPS Alerts_Alert Data',
    'IDS/IPS Alerts_No Alert Data',
    'Log Source_Firewall',
    'Log Source_Server',
    'Protocol_ICMP',
    'Protocol_TCP',
    'Protocol_UDP',
    'Packet Type_Control',
    'Packet Type_Data',
    'Traffic Type_DNS',
    'Traffic Type_FTP',
    'Traffic Type_HTTP',
    'Malware Indicators_IoC Detected',
    'Malware Indicators_No IoC Detected',
    'Alerts/Warnings_Alert Triggered',
    'Alerts/Warnings_No Alert Triggered',
    'Attack Signature_Known Pattern A', 
    'Attack Signature_Known Pattern B',
    'Action Taken_Blocked', 
    'Action Taken_Ignored', 'Action Taken_Logged',
    'Network Segment_Segment A', 
    'Network Segment_Segment B',
    'Network Segment_Segment C', 
    'Firewall Logs_Log Data',
    'Firewall Logs_No Log Data', 
    'IDS/IPS Alerts_Alert Data',
    'IDS/IPS Alerts_No Alert Data',
    'Log Source_Firewall',
    'Log Source_Server'
]

target_column = 'attack type'

### get the values of the features columns for the training data


In [15]:
X_train = df_train.loc[:,features].values
y_train = df_train['Attack Type'].values

### get the values of the columns for the test data

In [16]:
x_test = df_test.loc[:,features].values
y_test = df_test['Attack Type'].values

## Testing models 

our problem is a classification problem , we then tried the following classification algorithms :

1) Logistic Regression
2) Decision tree classifier
3) Random Forest Classifier
4) XGboost

## Logistic regression 

In [17]:
lr_model = LogisticRegression(random_state=42,max_iter=1500)

In [18]:
# here we train the model on the training data
lr_model.fit(X=X_train, y=y_train)

LogisticRegression(max_iter=1500, random_state=42)

In [22]:
cf = pd.DataFrame(
    columns=["y_test_DDoS","y_test_Intrusion","y_test_Malware"],index=["y_predicted_DDoS","y_predicted_Intrusion","y_predicted_Malware"]
)

cf.loc[:,:] = confusion_matrix(y_true= y_test,y_pred= y_test_predicted)
cf

,y_test_DDoS,y_test_Intrusion,y_test_Malware
y_predicted_DDoS,1126,625,885
y_predicted_Intrusion,1120,657,944
y_predicted_Malware,1033,643,967


## Decision Tree Classifier

## Random forest classifier

## Xgboost